In [1]:
import pandas as pd
import numpy as np


In [132]:
df = pd.read_pickle("./cinemahall_df.pkl")

In [133]:
df.shape

(6368, 25)

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6368 entries, 1 to 1
Data columns (total 25 columns):
Art Direction        3070 non-null object
Background Music     1449 non-null object
Banner               3992 non-null object
Choreography         117 non-null object
Cinematography       4601 non-null object
Costumes             646 non-null object
Design               2202 non-null object
Dialogue             4352 non-null object
Director             5935 non-null object
Distribution         1647 non-null object
Editing              4360 non-null object
Lyrics               5653 non-null object
Makeup               722 non-null object
Movie                6368 non-null object
Movie Genre          288 non-null object
Music                5874 non-null object
Original Story       112 non-null object
Producer             3950 non-null object
Screenplay           4519 non-null object
Shooting Location    308 non-null object
Singers              5395 non-null object
Sound Design        

In [138]:
df.isnull().sum()

Art Direction        3298
Background Music     4919
Banner               2376
Choreography         6251
Cinematography       1767
Costumes             5722
Design               4166
Dialogue             2016
Director              433
Distribution         4721
Editing              2008
Lyrics                715
Makeup               5646
Movie                   0
Movie Genre          6080
Music                 494
Original Story       6256
Producer             2418
Screenplay           1849
Shooting Location    6060
Singers               973
Sound Design         6256
Status                  0
Story                1924
cast                    0
dtype: int64

In [139]:
#ADDING YEAR to the column derived from the movie name.
import re
def get_year(text):
    pattern = re.compile(r"\((\d+)\)")
    year = ''.join(pattern.findall(text))
    try:
        if int(year)>2020:
            return "UNKNOWN"
        else:
            return int(year)
    except:
        return "UNKNOWN"
        

df["year"] = df.Movie.apply(get_year)
df.head()

,Art Direction,Background Music,Banner,Choreography,Cinematography,Costumes,Design,Dialogue,Director,Distribution,...,Original Story,Producer,Screenplay,Shooting Location,Singers,Sound Design,Status,Story,cast,year
1,NaN,NaN,Travancore National Pictures Ltd,NaN,JC Daniel,NaN,NaN,NaN,JC Daniel,NaN,...,NaN,JC Daniel,JC Daniel,NaN,NaN,NaN,1928-07-11,JC Daniel,"[JC Daniel, Johnson, PK Rosy, Sunder Raj]",1928
1,NaN,NaN,Sree Rajeswari Films,NaN,PE Nayik,NaN,NaN,NaN,PV Rao,NaN,...,NaN,R Sundararaj,PV Rao,NaN,NaN,NaN,1933-12-05,CV Raman Pillai,"[Pious, Padmini, AP Krishna Menon, Devaki Bhai...",1933
1,NaN,NaN,Modern Theatres,NaN,Bodo Gush Walker,NaN,NaN,Muthukulam Raghavan Pilla,S Nottani,NaN,...,NaN,TR Sundaram,Muthukulam Raghavan Pilla,NaN,"KK Aroor, Master Madanagopal, MK Kamalam, Pall...",NaN,1938-01-19,A Sundaram Pillai,"[Alleppey Vincent, MK Kamalam, KK Aroor]",1938
1,NaN,NaN,Shyamala Pictures,NaN,N Cooper,NaN,NaN,C Madhavan Pillai,S Nottani,NaN,...,NaN,Annamala Chettiyar,C Madhavan Pillai,NaN,"CK Rajam, KK Aroor, Mavelikkara L Ponnamma, PK...",NaN,1940-07-04,C Madhavan Pillai,"[CA Seethalakshmi, Nanukkuttan, Mavelikkara L ...",1940
1,NaN,NaN,Madras United Artist Corporation,NaN,NaN,NaN,NaN,NP Chellappan Nair,K Subramaniam,NaN,...,Mythology,NaN,NaN,NaN,"Papanasam Sivan, VA Chellappa, Kumari Lakshmi",NaN,1941-08-17,NaN,"[Guru Gopinath, Kumari Lakshmi, NP Chellappan ...",1941


In [141]:
#normalizing column names
columns = [i.lower() for i in df.columns]
df.columns = columns

In [143]:
#Rearranging columns
columns =['year','movie', 'movie genre','director', 'distribution', 'editing','producer','screenplay',
          'music','art direction', 'background music', 'banner', 'choreography', 'cinematography', 'costumes', 
          'design', 'dialogue',  'lyrics', 'makeup', 'original story','shooting location', 'singers', 'sound design', 'status',
          'story','cast']
df = df[columns]
df.shape

(6368, 26)

In [144]:
#removing year with unknown release data / All are dubbed movies
df = df[df["year"] != "UNKNOWN" ]
df.shape

(6297, 26)

In [600]:
# mov1 = list(df.movie)
df.to_pickle("./mov1.pkl")

In [147]:
#Grouping by year
df_yearcount = df[["movie","year"]].groupby("year").count()
df_yearcount =df_yearcount.reset_index()
df_yearcount.tail()

,year,movie
73,2016,183
74,2017,229
75,2018,264
76,2019,297
77,2020,123


In [108]:
#converting to json for visualization
dfjson1 = df_yearcount.to_json(orient = "split")

In [333]:
#top50 director
dir_grp = df.director.value_counts()
dir_grp = dir_grp.reset_index()
dir_grp.columns=["director","count"]
dir_grp = dir_grp[:25]
dir_grp.shape

(25, 2)

In [334]:
dir_names = list(dir_grp.director)

In [335]:
ddf = df[["year","movie","director"]]
ddf = ddf[ddf.director.isin(dir_names)]
dirdata = ddf.groupby(["year","director"]).count()
dirdata = dirdata.reset_index()
len(dirdata.director.value_counts())

25

In [592]:
dirjson =[]
import math

for i in range(25):
    temp = {}
    data =[]
    temp["label"] = dir_names[i]
    temp["backgroundColor"] = colors[i]
    
    k = dirdata[dirdata.director == dir_names[i]]
    k2 = pd.merge(diryears,k,on="year",how='outer')
    mylist = [0 if math.isnan(x) else int(x) for x in list(k2.movie)]
    temp["data"] =mylist

            
    
    
    dirjson.append(temp)

dirjson
                 

[{'label': 'Sasikumar', 'backgroundColor': '#2A5784', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 3, 2, 1, 2, 4, 7, 4, 11, 7, 15, 10, 6, 5, 5, 8, 9, 3, 6, 6, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'label': 'IV Sasi', 'backgroundColor': '#F59D3D', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 12, 9, 5, 6, 4, 7, 5, 7, 5, 4, 4, 4, 2, 3, 3, 2, 3, 1, 0, 0, 2, 0, 1, 1, 1, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'label': 'Joshiy', 'backgroundColor': '#FFC686', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 5, 5, 9, 7, 6, 2, 3, 3, 3, 0, 1, 1, 1, 1, 0, 2, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 1, 1, 0, 2, 1, 3, 1, 1, 0, 0, 0, 1, 0]}, {'label': 'M Krishnan Nair', 'backgroundColor': '#BE89AC', 'data': [0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 3, 4, 5, 6, 6, 3, 4, 2, 1, 3, 1, 0, 2, 4, 5, 4, 3, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [591]:
palette =['#2A5784', '#43719F', '#5B8DB8', '#7AAAD0', '#9BC7E4', '#BADDF1', '#E1575A', '#EE7423', '#F59D3D', '#FFC686', '#9D7760', '#F1CF63', '#7C4D79', '#9B6A97', '#BE89AC', '#D5A5C4', '#EFC9E6', '#BBB1AC', '#24693D', '#398949', '#61AA57', '#7DC470', '#B4E0A7', '#dbf7d3', '#565d47']

colors =['#2A5784', '#F59D3D','#FFC686', '#BE89AC','#61AA57','#43719F', '#BADDF1','#9D7760','#D5A5C4','#7DC470','#5B8DB8', '#EFC9E6','#F1CF63','#BBB1AC','#B4E0A7',
         '#7AAAD0', '#E1575A','#7C4D79','#24693D','#dbf7d3','#9BC7E4', '#EE7423','#9B6A97','#398949','#565d47']
  
colors
       

['#2A5784', '#F59D3D', '#FFC686', '#BE89AC', '#61AA57', '#43719F', '#BADDF1', '#9D7760', '#D5A5C4', '#7DC470', '#5B8DB8', '#EFC9E6', '#F1CF63', '#BBB1AC', '#B4E0A7', '#7AAAD0', '#E1575A', '#7C4D79', '#24693D', '#dbf7d3', '#9BC7E4', '#EE7423', '#9B6A97', '#398949', '#565d47']

In [201]:
temp =list(banner["count"][:50])
temp

[58, 48, 36, 28, 25, 24, 20, 20, 19, 18, 18, 18, 17, 16, 16, 16, 16, 15, 15, 14, 14, 13, 13, 13, 13, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 8, 8]

In [365]:
#top50 Banner
banner = df.banner.value_counts()
banner = banner.reset_index()
banner.columns=["banner","count"]
banner = banner[:25]
banner

,banner,count
0,Neela Productions,58
1,Sunitha Productions,48
2,Excel Productions,36
3,Jaya Maruthi,28
4,Udaya,25
5,Manjilas,24
6,Supriya,20
7,Century Films,20
8,Grihalakshmi Productions,19
9,Seven Arts International LTD,18


In [373]:
banner_names = list(banner.banner)
bdf = df[["year","movie","banner"]]
bdf = bdf[bdf.banner.isin(banner_names)]
bandata = bdf.groupby(["year","banner"]).count()
bandata = bandata.reset_index()
len(bandata.banner.value_counts())
bdf.head(3)

,year,movie,banner
1,1952,Achan (1952),Excel Productions
1,1952,Aathmasakhi (1952),Neela Productions
1,1953,Ponkathir (1953),Neela Productions


In [375]:
bff = pd.DataFrame(df)
df_56= (df[(1950 < df.year) & (df.year <= 1960 )])
df_67= (df[(1960 < df.year) & (df.year <= 1970 )])
df_78= (df[(1970 < df.year) & (df.year <= 1980 )])
df_89= (df[(1980 < df.year) & (df.year <= 1990 )])
df_90= (df[(1990 < df.year) & (df.year <= 2000 )])
df_01= (df[(2000 < df.year) & (df.year <= 2010 )])
df_12= (df[(2010 < df.year) & (df.year <= 2020 )])




,year,movie,movie genre,director,distribution,editing,producer,screenplay,music,art direction,...,dialogue,lyrics,makeup,original story,shooting location,singers,sound design,status,story,cast
1,1951,Jeevithanouka (1951),NaN,K Vembu,NaN,KD George,"M Kunchacko, KV Koshy",Muthukulam Raghavan Pilla,V Dakshinamoorthy,NaN,...,Muthukulam Raghavan Pilla,"Abhayadev, Vallathol",NaN,NaN,NaN,"V Dakshinamoorthy, P Leela, Alappuzha Pushpam,...",NaN,1951-03-15,Muthukulam Raghavan Pilla,"[BS Saroja, Thikkurissi Sukumaran Nair]"
1,1951,Rakthabandham (1951),NaN,MR Vittal,NaN,TM Lal,NK Karunakaran Pillai,K Chellappan Pillai,"SM Subbaiah Naidu, SN Chaami (SN Ranganathan)",NaN,...,K Chellappan Pillai,"Abhayadev, Thumbaman Padmanabhan Kutty",NaN,NaN,NaN,"P Leela, Kaviyoor CK Revamma, Vaikkom Mani, Ku...",NaN,1951-05-18,K Chellappan Pillai,"[Ambalappuzha Meenakshi, Cherthala Vasudeva Ku..."
1,1951,Yaachakan (1951),NaN,R Velappan Nair,NaN,KD George,KS Akhileswarayyar,PS Nair,SN Chaami (SN Ranganathan),A Balu,...,"Abhayadev, PS Nair","Abhayadev, Bodheswaran, G Sankara Kurup",NaN,NaN,NaN,"Kaviyoor CK Revamma, T Lokanathan, VN Rajan",NaN,1951-12-09,PS Nair,"[Thiruvananthapuram Kuttan Pillai, Muthukulam ..."
1,1951,Kerala Kesari (1951),NaN,V Krishnan,Geo Release,NaN,Vaikkom Vasudevan Nair,N Shankarapillai,Jnanamani,NaN,...,"KN Gopalan Nair, N Shankarapillai",Thumbaman Padmanabhan Kutty,NaN,NaN,NaN,"P Leela, Kaviyoor CK Revamma, Mothi, Thankam V...",NaN,1951-05-17,VK Kumar,[Durga Varma]
1,1951,Navalokam (1951),NaN,V Krishnan,NaN,KD George,Pappachan,Ponkunnam Varkey,V Dakshinamoorthy,NaN,...,Ponkunnam Varkey,P Bhaskaran,NaN,NaN,NaN,"V Dakshinamoorthy, P Leela, Alappuzha Pushpam,...",NaN,1951-03-29,Ponkunnam Varkey,"[Thikkurissi Sukumaran Nair, Muthukulam Raghav..."


In [371]:
banyear =pd.DataFrame({"year": list(bandata.year)})
list(banyear.year.unique())

[1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [368]:
banj =[]
import math

for i in range(25):
    temp = {}
    data =[]
    temp["label"] = banner_names[i]
    temp["backgroundColor"] = colours[i]
    
    k = bandata[bandata.banner == banner_names[i]]
    k2 = pd.merge(banyear,k,on="year",how='outer')
    mylist = [0 if math.isnan(x) else int(x) for x in list(k2.movie)]
    temp["data"] =mylist

            
    
    
    banj.append(temp)

banj

[{'label': 'Neela Productions', 'backgroundColor': '#2A5784', 'data': [1, 1, 1, 2, 2, 3, 3, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
#top50 cinematography
cinematography = df.cinematography.value_counts()
cinematography = cinematography.reset_index()
cinematography.columns=["music","count"]
cinematography = cinematography[:50]
cinematography

In [380]:
#top50 editors
editors = df.editing.value_counts()
editors = editors.reset_index()
editors.columns=["editing","count"]
editors50 = editors[:50]
len(editors.editing.unique())

549

In [ ]:
#top50 producers
producers = df.producer.value_counts()
producers = producers.reset_index()
producers.columns=["producers","count"]
producers = producers[:50]
producers

In [ ]:
#top50 screenplay
screenplay = df.screenplay.value_counts()
screenplay = screenplay.reset_index()
screenplay.columns=["screenplay","count"]
screenplay = screenplay[:100]
screenplay.head(25)

In [384]:
#top50 story
story = df.story.value_counts()
story = story.reset_index()
story.columns=["story","count"]
story50 = story[:50]
print(story.shape[0])
story.head(5)

1899


,story,count
0,MT Vasudevan Nair,55
1,SL Puram Sadanandan,51
2,Sreekumaran Thampi,49
3,Lohithadas,46
4,Ranjith,45


In [388]:
#art direction
art = df["art direction"].value_counts()
art = art.reset_index()
art.columns=["art","count"]
art50 = story[:50]
print(art.shape[0])
art.head(5)

602


,art,count
0,S Konnannattu,162
1,Radhakrishnan (RK),93
2,Boban,92
3,K Krishnankutty,81
4,K Balan,69


In [390]:
#lyrics  #TOP10 visualize
lyrics = df["lyrics"].value_counts()
lyrics = lyrics.reset_index()
art.columns=["lyrics","count"]
art50 = lyrics[:50]
print(lyrics.shape[0])
lyrics.head(10)

2067


,index,lyrics
0,Poovachal Khader,276
1,Kaithapram,247
2,Gireesh Puthenchery,234
3,Sreekumaran Thampi,222
4,Bichu Thirumala,217
5,P Bhaskaran,208
6,ONV Kurup,181
7,Vayalar,178
8,Mankombu Gopalakrishnan,155
9,Yusufali Kecheri,105


In [397]:
#cast
totcast = set(df["cast"].sum())
len(totcast)

3709

In [398]:
actor_df = pd.DataFrame(columns=totcast)
actor_df

,Sinbad,MA Nair,Sreesankhya,Tarun Kumar,CM Abraham,Latha Thomas,Shruthi (Kannada),Nithi Singh,Ajin Shaji,Ajith Kumar,...,Pankajavalli,Stephen,Madhu Attukal,Manoj Guiness,Valsa James,Jyothi Meena,Aswathy Nair,Mohanraj (Old),Lalitha,Madhubala (Old)


In [429]:
castdf = df[["year","movie","cast"]]
castdf = castdf.reset_index(drop =True)
castdf.shape

(6297, 3)

In [430]:
for i in totcast:
    castdf[i] = 0


In [431]:
castdf.shape

(6297, 3712)

In [432]:

for i, row in castdf.iterrows():
    for j in castdf["cast"][i]:
        castdf.at[i,j] = 1

In [437]:
castdf

,year,movie,cast,Sinbad,MA Nair,Sreesankhya,Tarun Kumar,CM Abraham,Latha Thomas,Shruthi (Kannada),...,Pankajavalli,Stephen,Madhu Attukal,Manoj Guiness,Valsa James,Jyothi Meena,Aswathy Nair,Mohanraj (Old),Lalitha,Madhubala (Old)
0,1928,Vigathakumaaran (1928),"[JC Daniel, Johnson, PK Rosy, Sunder Raj]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [483]:
cast_list = list(totcast)
y_dummies = castdf[cast_list].values
y_max = np.where(y_dummies[0] == 1)
[cast_list[i] for i in list(y_max[0])]


['JC Daniel', 'Sunder Raj', 'Johnson', 'PK Rosy']

In [ ]:
actor_freq = castdf.sum().to_frame()
castdf

In [492]:
actor_freq = actor_freq[3:]

In [498]:
actor_freq.columns=actor_freq.columns.astype(str)

In [501]:
actor_freq = actor_freq.reset_index()
actor_freq.columns = ["actor","count"]


In [505]:
actor_freq = actor_freq.sort_values("count",ascending=False)


In [509]:
#actor_freq.reset_index()
actor_freq = actor_freq[["actor","count"]]

In [510]:
actor_freq.head()

,actor,count
617,Prem Nazir,500
2794,Mammootty,367
504,Mohanlal,309
1830,Madhu,290
121,Jayabharathi,263


In [529]:
m_max = np.where(mohanlal == 1)
for i in

309

In [535]:
mohanlal = castdf["Mohanlal"].values
m_max = np.where(mohanlal == 1)
[castdf["movie"].iloc[i] for i in list(m_max[0])]
castdf

['Thiranottam (1978)', 'Manjil Virinja Pookkal (1980)', 'Dhruvasangamam (1981)', 'Oothikkaachiya Ponnu (1981)', 'Dhanya (1981)', 'Football (1982)', 'Enthino Pookkunna Pookkal (1982)', 'Aa Divasam (1982)', 'Kaaliyamarddanam (1982)', 'Enikkum Oru Divasam (1982)', 'Kelkkaatha Shabdam (1982)', 'Sindoorasandhyaykku Mounam (1982)', 'Njaanonnu Parayatte (1982)', 'Ente Kadha (1983)', 'Himavaahini (1983)', 'Engane Nee Marakkum (1983)', 'Arabikkadal (1983)', 'Iniyenkilum (1983)', 'Naanayam (1983)', 'Chakravaalam Chuvannappol (1983)', 'Sandhyaykku Virinja Poovu (1983)', 'Asthram (1983)', 'Kolakomban (1983)', 'Hello Madras Girl (1983)', 'Oru Mukham Pala Mukham (1983)', 'Changaatham (1983)', 'Kaattathe Kilikkoodu (1983)', 'Aattakkalaasham (1983)', 'Naseema (Thamburu) (1983)', 'Gurudakshina (1983)', 'Visa (1983)', 'Arunayude Prabhaatham (1983)', 'Ivide Thudangunnu (1984)', 'Uyarangalil (1984)', 'Thirakal (1984)', 'Kurishuyudham (1984)', 'Appunni (1984)', 'Poochaykkoru Mookkuthi (1984)', 'Kaliyil Alp

In [544]:
castdf

,year,movie,cast,Sinbad,MA Nair,Sreesankhya,Tarun Kumar,CM Abraham,Latha Thomas,Shruthi (Kannada),...,Pankajavalli,Stephen,Madhu Attukal,Manoj Guiness,Valsa James,Jyothi Meena,Aswathy Nair,Mohanraj (Old),Lalitha,Madhubala (Old)
0,1928,Vigathakumaaran (1928),"[JC Daniel, Johnson, PK Rosy, Sunder Raj]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1933,Maarthandavarma (1933),"[Pious, Padmini, AP Krishna Menon, Devaki Bhai...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1938,Balan (1938),"[Alleppey Vincent, MK Kamalam, KK Aroor]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1940,Jnaanaambika (1940),"[CA Seethalakshmi, Nanukkuttan, Mavelikkara L ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1941,Prahlaada (1941),"[Guru Gopinath, Kumari Lakshmi, NP Chellappan ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1948,Nirmala (1948),"[PJ Cheriyan, Baby Joseph, Cherthala Vasudeva ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1949,Vellinakshathram (1949),"[Gayaka Peethambaram, Lalitha Devi]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1950,Sthree (1950),"[Thikkurissi Sukumaran Nair, CS Radhadevi, Vai...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1950,Prasanna (1950),[TS Balayya],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1950,Nallathanka (1950),"[Augustine Joseph, Muthukulam Raghavan Pilla, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [545]:
mmov = castdf[castdf.Mohanlal == 1]["movie"].str.replace(r"\(.*\)","")

In [539]:
mwiki = ["Ram",
"Marakkar: Arabikadalinte Simham ",
"Drishyam 2 ",
"Big Brother",
"Lucifer",
"Ittymaani: Made in China",
"Odiyan",
"Neerali",
"Kayamkulam Kochunni",
"Drama",
"Aadhi",
"Villain",
"Velipadinte Pusthakam",
"Munthirivallikal Thalirkkumbol",
"1971: Beyond Borders",
"Pulimurugan",
"Oppam",
"Rasam",
"Loham",
"Lailaa O Lailaa",
"Kanal",
"Ennum Eppozhum",
"Peruchazhi",
"Mr. Fraud",
"Koothara",
"Red Wine",
"Lokpal",
"Ladies and Gentleman",
"Kadal Kadannu Oru Maathukutty",
"Geethaanjali",
"Drishyam",
"Spirit",
"Run Baby Run",
"Karmayodha",
"Grandmaster",
"Casanovva",
"Snehaveedu",
"Pranayam",
"Oru Marubhoomikkadha",
"Christian Brothers",
"China Town",
"Shikkar",
"Oru Naal Varum",
"Kandahar",
"Janakan",
"Alexander the Great",
"Sagar Alias Jacky Reloaded",
"Red Chillies",
"Evidam Swargamanu",
"Bhramaram",
"Bhagavan",
"Angel John",
"Twenty:20",
"Pakal Nakshatrangal",
"Mizhikal Sakshi",
"Madampi",
"Kurukshetra",
"Innathe Chintha Vishayam",
"College Kumaran",
"Aakasha Gopuram",
"Rock N' Roll",
"Paradesi",
"Hallo",
"Flash",
"Chotta Mumbai",
"Alibhai",
"Vadakkumnathan",
"Rasathanthram",
"Photographer",
"Mahasamudram",
"Kilukkam Kilukilukkam",
"Keerthi Chakra",
"Baba Kalyani",
"Udayon",
"Udayananu Tharam",
"Thanmathra",
"Naran",
"Chandrolsavam",
"Wanted",
"Vismayathumbathu",
"Vamanapuram Bus Route",
"Natturajavu",
"Mampazhakkalam",
"Thiranottam",
"Mr. Brahmachari",
"Kilichundan Mampazham",
"Hariharan Pillai Happy Aanu",
"Balettan",
"Thandavam",
"Onnaman",
"Chathurangam",
"Unnathangalil",
"raavanaprabhu",
"Praja",
"Kakkakuyil",
"Achaneyanenikkishtam",
"Sraddha",
"Narasimham",
"Life is Beautiful",
"Devadoothan",
"Vanaprastham",
"usthaad",
"Olympiyan Anthony Adam",
"Summer in Bethlahem",
"Rakthasakshikal Sindabad",
"Kanmadam",
"Harikrishnans",
"Ayal Kadha Ezhuthukayanu",
"Varnapakittu",
"Oru Yathramozhi",
"Guru",
"Chandralekha",
"Aaraam Thampuran",
"The Prince",
"Kaalapani",
"Thacholi Varghese Chekavar",
"Sphadikam",
"Nirnayam",
"Manthrikam",
"Agnidevan",
"Thenmavin Kombath",
"Pingami",
"Pavithram",
"Pakshe",
"Minnaram",
"Mithunam",
"Maya Mayooram",
"Manichithrathazhu",
"kalippaattam",
"Gandharvam",
"Devaasuram",
"Chenkol",
"Butterflies",
"Yoddha",
"Vietnam Colony",
"Soorya Gayathri",
"Sadayam",
"Rajashilpi",
"Naadody",
"Kamaladalam",
"Aham",
"Adhwaytham",
"Vishnulokam",
"Vasthuhara",
"Uncle Bun",
"Ulladakkam",
"Kizhakkunarum Pakshi",
"Kilukkam",
"Dhanam",
"Bharatham",
"Abhimanyu",
"Thazhvaram",
"No.20 Madras Mail",
"Mukham",
"Lal Salam",
"Kadathanadan Ambadi",
"Indrajaalam",
"His Highness Abdullah",
"Aye Auto",
"Arhatha",
"Appu",
"Akkare Akkare Akkare",
"Varavelpu",
"Vandanam",
"Season",
"Peruvannapurathe Visheshangal",
"Naduvazhikal",
"Lal Americayil",
"Kireedam",
"Douthyam",
"dasaratham",
"Adhipan",
"Vellanakalude Nadu",
"Ulsavapittennu",
"Pattanapravesham",
"Paadha Mudra",
"Orkkappurathu",
"Mukunthetta Sumitra Vilikkunnu",
"Moonnam Mura",
"Manu Uncle",
"Chithram",
"Ayitham",
"Aryan",
"Anuragi",
"Vazhiyorakkazhcakal",
"Unnikale Oru Kadha Parayam",
"Thoovanathumbikal",
"Sarvakalashala",
"Nadodikkattu",
"Kaiyethum Doorathu",
"January Oru Orma",
"Ivide Ellavarkum Sugham",
"Irupatham Noottandu",
"Cheppu",
"Bhoomiyile Rajakkanmar",
"Amrutham Gamaya",
"Adimakal Udamakal",
"Yuvajanotsavam",
"Vartha",
"Thalavattam",
"T. P. Balagopalan M.A.",
"Sukhamo Devi",
"Shobaraj",
"Sanmanassullavarkku Samadhanam",
"Revathikkoru Pavakkutty",
"Rajavinte Makan",
"Poomukhappadiyil Ninneyum Kathu",
"Pappan Priyappetta Pappan",
"Panchagni",
"Padayani",
"Oppam Oppathinoppam",
"Onnu Muthal Poojyam Vare",
"Ninnishtam Ennishtam",
"Nimishangal",
"Neram Pularumbol",
"Namukku Parkkan Munthirithoppukal",
"Mizhineerppoovukal",
"Mazha Peyyunnu Maddalam Kottunnu",
"Manasilloru Manimuthu",
"Kunjattakkilikal",
"Kaveri",
"Kariyilakkattupole",
"Iniyum Kurukshetrum",
"Hello My Dear Wrong Number",
"Geetham",
"Gandhinagar 2nd Street",
"Ente Entethu Mathram",
"Doore Doore Oru Koodu Koottam",
"Deshadanakkili Karayarilla",
"Adiverukal",
"Abhayam Thedi",
"Vasantha Sena",
"Uyarum Njan Nadake",
"Rangam",
"Pathamudayam",
"Parayanumvayya Parayathirikkanumvayya",
"Ormikkaan Omanikkaan",
"Onnanam Kunnil Oradi Kunnil",
"Nokketha Doorathu Kannum Nattu",
"Njan Piranna Nattil",
"Nayakan",
"Mulamoottil Adima",
"Koodum Thedi",
"Karimpinpoovinakkare",
"Kandu Kandarinju",
"Jeevante Jeevan",
"Idanilangal",
"Guruji Oru Vakku",
"Ezhu Muthal Onpathu Vare",
"Boeing Boeing",
"Azhiyatha Bandhangal",
"Avidathepole Ivideyum",
"Aram + Aram = Kinnaram",
"Anubandham",
"Angadikkappurathu",
"Adhyayam Onnu Muthal",
"Vetta",
"Vanitha Police",
"Uyarangalil",
"Unaru",
"Thirakal",
"Swanthamevide Bandhamevide",
"Sreekrishna Parunthu",
"Poochakkoru Mookkuthi",
"Pavam Poornima",
"Oru Kochu Swapnam",
"Onnanu Nammal",
"Manasariyathe",
"Lakshmana Rekha",
"Kurishuyudham",
"Kilikkonchal",
"Kaliyil Alpam Karyam",
"Ivide Thudangunnu",
"Itha Innu Muthal",
"Athirathram",
"Ariyaatha Veethikal",
"Appunni",
"Akkare",
"Aduthaduthu",
"Adiyozhukkukal",
"Aalkkoottathil Thaniye",
"Visa",
"Thaavalam",
"Sesham Kazhchayil",
"Sandhyakku Virinja Poovu",
"Pinnilavu",
"Oru Mukham Pala Mukham",
"Naseema",
"Nanayam",
"Marakkillorikkalum",
"Kuyiline Thedi",
"Kolakkomban",
"Kattathe Kilikkoodu",
"Iniyengilum",
"Himavahini",
"Hello Madras Girl",
"Guru Dakshina",
"Ente Mamattikkuttiyammakku",
"Ente Katha",
"Engane Nee Marakkum",
"Changatham",
"Chakravalam Chuvannappol",
"Bhookambam",
"Attakkalasam",
"Asthram",
"Arabikkadal",
"Aadhipathyam",
"Sree Ayyappanum Vavarum",
"Sindoora Sandhyakku Mounam",
"Padayottam",
"Njan Onnu Parayatte",
"Madrasile Mon",
"Kurukkante Kalyanam",
"Kelkkaatha Sabdham",
"Kaliyamardanam",
"Football",
"Enthino Pookunna Pookkal",
"Ente Mohangal Poovaninju",
"Enikkum Oru Divasam",
"Aakrosham",
"Aa Divasam",
"Thenum Vayambum",
"Thakilu Kottampuram",
"Sanchari",
"Oothikachiya Ponnu",
"Dhruvasangamam",
"Dhanya",
"Attimari",
"Ahimsa"]

In [570]:
dif = list(set(mwiki)-set(mmov))

In [552]:
mmov =list(mmov.values)

In [554]:
mmov = [i.lower().strip() for i in mmov]

In [555]:
mwiki =[i.lower().strip() for i in mwiki]

In [557]:
len(mwiki)

329

In [558]:
len(mmov)

309

In [581]:
import Levenshtein as lev
n =[]
for i in dif:
    for j in mmov:
        if lev.ratio(j,i)>0.9:
            print(j);n.append(i)



ulsavappittennu
onnaaman
vaasthuhaara
vazhiyorakkaazhchakal
vaamanapuram bus route
olympian antony adam
kelkkaatha shabdam
bhoomiyile raajakkanmaar
munthirivallikal thalirkkumpol
minnaaram
kalippaattam
ittymaani made in china
iniyum kurukshetram
keerthichakra
yuvajanolsavam
sindoorasandhyaykku mounam
thaazhvaaram
sarvakalashaala
athiraathram
neeraali
himavaahini
angaadikkappurathu
thacholi vargheese chekavar
kaattathe kilikkoodu
raajashilpi
kaaveri
guruji oru vaakku
his highness abdulla
bhagavaan
maanthrikam
sreekrishnapparunthu
poochaykkoru mookkuthi
naaduvaazhikal
dasaratham
ezhumuthal onpathuvare
changaatham
ividam swargamaanu
namukku paarkkaan munthirithoppukal
mr fraud
rock n roll
manassariyaathe
paadamudra
karimbinpoovinakkare
adhyaayam onnumuthal
iniyenkilum
oothikkaachiya ponnu
pakal nakshathrangal
college kumaaran
gaandharvam
chakravaalam chuvannappol
marakkar arabikadalinte simham
naayakan
kolakomban
aakaashagopuram
kunjaattakkilikal
aaryan
velipaadinte pusthakam
panchaagni
k

In [582]:
set(dif)-set(n)

{'ormikkaan omanikkaan', 'ahimsa', 'bhookambam', 'sanchari', 'mizhikal sakshi', 'oru kochu swapnam', 'aalkkoottathil thaniye', 'naadody', 'kandahar', 'no.20 madras mail', 'peruvannapurathe visheshangal', 'vanitha police', 'aadhi', 'madampi', 'aakrosham', 'sree ayyappanum vavarum', 'parayanumvayya parayathirikkanumvayya', 'kayamkulam kochunni', 'maya mayooram', 'attakkalasam', 'ariyaatha veethikal', 'ente mohangal poovaninju', 'twenty:20', 'adhwaytham', 'achaneyanenikkishtam', 'mithunam', 'thakilu kottampuram', 'unaru', 'lal salam', 'thenum vayambum', 't. p. balagopalan m.a.', 'aadhipathyam', 'kadal kadannu oru maathukutty', 'sesham kazhchayil', 'padayottam', 'thaavalam', 'itha innu muthal', 'natturajavu', 'oru marubhoomikkadha', 'thenmavin kombath', 'mampazhakkalam', 'manu uncle', 'pinnilavu', 'shobaraj', 'wanted', 'ente katha', 'aduthaduthu', 'marakkillorikkalum', 'kurukkante kalyanam', 'madrasile mon', 'unnathangalil', 'ayal kadha ezhuthukayanu', 'aram + aram = kinnaram', 'summer in 

In [587]:
castdf[castdf.movie == 'Ente Mohangal Poovaninju (1982)']

,year,movie,cast,Sinbad,MA Nair,Sreesankhya,Tarun Kumar,CM Abraham,Latha Thomas,Shruthi (Kannada),...,Pankajavalli,Stephen,Madhu Attukal,Manoj Guiness,Valsa James,Jyothi Meena,Aswathy Nair,Mohanraj (Old),Lalitha,Madhubala (Old)
1306,1982,Ente Mohangal Poovaninju (1982),"[Menaka Sureshkumar, Shanker]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
